# Run/Tumble TD with runner (same simulation)

Companion to the manual run_tumble_td demo. Uses runner.stream to step the exact same environment and policy, and plots the resulting trajectory and action statistics.

In [ ]:
# Path and plotting setup
import sys, pathlib

repo_root = pathlib.Path.cwd()
if not (repo_root / "src").exists():
    repo_root = repo_root.parent
sys.path.insert(0, str(repo_root / "src"))

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats

mpl.use("module://matplotlib_inline.backend_inline")
set_matplotlib_formats("png")
from IPython.display import display

import plume_nav_sim as pns
from plume_nav_sim.policies.run_tumble_td import RunTumbleTemporalDerivativePolicy
from plume_nav_sim.runner import runner as r

In [ ]:
# Same configuration as the manual demo
grid_size = (64, 64)
source_location = (48, 48)
start_location = (16, 16)
max_steps = 500
seed = 123

env = pns.make_env(
    grid_size=grid_size,
    source_location=source_location,
    start_location=start_location,
    max_steps=max_steps,
    action_type="run_tumble",
    observation_type="concentration",
    reward_type="step_penalty",
    render_mode="rgb_array",
)

policy = RunTumbleTemporalDerivativePolicy(threshold=1e-6, eps_seed=seed)

# Collect events via runner.stream
positions = []
actions = []
concs = []
dcs = []
last_c = None
total_reward = 0.0
final_frame = None
for ev in r.stream(env, policy, seed=seed, render=True):
    actions.append(int(ev.action))
    c = float(ev.obs[0])
    concs.append(c)
    if last_c is None:
        dcs.append(0.0)
    else:
        dcs.append(c - last_c)
    last_c = c
    if isinstance(ev.info, dict) and "agent_xy" in ev.info:
        positions.append(tuple(ev.info["agent_xy"]))
    total_reward += float(ev.reward)
    if ev.frame is not None:
        final_frame = ev.frame

print("Steps:", len(actions), "Total reward:", total_reward)

# Trajectory overlay
frame = final_frame if isinstance(final_frame, np.ndarray) else env.render("rgb_array")
grid_w, grid_h = grid_size
sx, sy = source_location
fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(frame)
ax.set_xlim(0, grid_w)
ax.set_ylim(grid_h, 0)
if positions:
    xs = [p[0] for p in positions]
    ys = [p[1] for p in positions]
    ax.plot(xs, ys, "-o", color="yellow", markersize=2, linewidth=1)
    ax.scatter([xs[0]], [ys[0]], c="lime", s=36, marker="^", label="start")
    ax.scatter([xs[-1]], [ys[-1]], c="magenta", s=30, label="end")
ax.scatter(
    [sx],
    [sy],
    marker="s",
    s=60,
    facecolors="none",
    edgecolors="red",
    linewidths=1.5,
    label="source",
)
ax.legend(loc="upper right")
ax.set_title("Run/Tumble TD with runner: final frame with trajectory")
display(fig)
plt.close(fig)

In [ ]:
# Action counts and time series
run_count = int(np.sum(np.array(actions) == 0))
tumble_count = int(np.sum(np.array(actions) == 1))
print("RUN:", run_count, "TUMBLE:", tumble_count)

fig, ax = plt.subplots(1, 2, figsize=(10, 3))
ax[0].bar(["RUN", "TUMBLE"], [run_count, tumble_count], color=["#4caf50", "#ff9800"])
ax[0].set_title("Action counts")
ts = np.arange(len(concs))
ax[1].plot(ts, concs, label="c")
ax[1].plot(ts, dcs, label="dc")
ax[1].set_title("Concentration and dC")
ax[1].legend()
plt.tight_layout()
display(fig)
plt.close(fig)
env.close()